<div class="container">
    <div class="components">
        <!-- queue header and your info like ETA -->
        <div class="div1">
            <div class="queue-header">
                <h2 id="teacherName">Mr. Mortensen's Queue</h2>
            </div>
            <div class="queue-info">
                <p id="position">Your position in line: #</p>
                <p id="estimatedTime">Estimated time: ~ mins</p>
            </div>
            <hr>
        </div>
        <!-- students in line -->
        <div class="div2">
            <div class="queue-list">
                <p>Students currently in line:</p>
                <ul id="studentList">
                    <!-- Students will be listed here -->
                </ul>
            </div>
            <div class="queue-buttons flex flex-wrap gap-2 justify-center mt-4">
                <button type="button" class="min-w-[12rem] h-12 px-4 py-2 text-center flex items-center justify-center bg-blue-600 text-white rounded-md" onclick="window.addToQueue()">Add Me to Queue</button>
                <button type="button" class="min-w-[12rem] h-12 px-4 py-2 text-center flex items-center justify-center bg-blue-600 text-white rounded-md" onclick="window.removeFromQueue()">Remove Me from Queue</button>
                <button type="button" class="min-w-[12rem] h-12 px-4 py-2 text-center flex items-center justify-center bg-blue-600 text-white rounded-md" onclick="window.location.href='{{site.baseurl}}/bathroom/issues'">Issues</button>
                <button type="button" class="min-w-[12rem] h-12 px-4 py-2 text-center flex items-center justify-center bg-blue-600 text-white rounded-md" onclick="window.location.href='{{site.baseurl}}/student'">Exit</button>
            </div>
        </div>
    </div>
    
</div>



<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script type="module" src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.esm.js"></script>
<script nomodule src="https://unpkg.com/ionicons@7.1.0/dist/ionicons/ionicons.js"></script>

<script type="module">
    import {javaURI, fetchOptions} from "{{site.baseurl}}/assets/js/api/config.js";
    window.addEventListener("load", () => {
        fetchUser();
    });

    async function fetchUser() {
        try {
            const response = await fetch(javaURI + "/api/person/get", {
                method: "GET",
                cache: "no-cache",
                credentials: "include",
                headers: { 
                    "Content-Type": "application/json",
                    "X-Origin": "client",
                },
            });
            
            if (response.ok) {
                const userInfo = await response.json();
                window.studentName = userInfo.name;
                window.sid = userInfo.sid;
                window.roles = userInfo.roles.map(role => role.name).join(", ");
                console.log("Person: ", userInfo);
                
                // After fetching user info, update UI elements and fetch queue data
                updateUIWithUserInfo();
                window.fetchQueueData();
            } else {
                // Handle login failure
                console.error("Failed to fetch user info. Status:", response.status);
                showLoginMessage();
            }
        } catch (error) {
            console.error("Error fetching user data:", error);
            showLoginMessage();
        }
    }

    function showLoginMessage() {
        // Display login message at the top of the page
        const queueHeader = document.querySelector(".queue-header");
        const loginMessage = document.createElement("div");
        loginMessage.className = "login-message";
        loginMessage.innerHTML = '<p style="color: red; font-weight: bold;">Please log in to join the queue</p>';
        queueHeader.after(loginMessage);
        
        // Disable queue buttons
        const buttons = document.querySelectorAll(".queue-buttons button");
        buttons.forEach(button => {
            if (button.innerText === "Add Me to Queue" || button.innerText === "Remove Me from Queue") {
                button.disabled = true;
                button.classList.add("disabled");
            }
        });
    }

    function updateUIWithUserInfo() {
        // Update any UI elements that need the user's info
        if (window.studentName) {
            // Example: You could add user info to the header
            const queueInfo = document.querySelector(".queue-info");
            const userInfo = document.createElement("p");
            userInfo.textContent = `Logged in as: ${window.studentName}`;
            queueInfo.appendChild(userInfo);
        }
    }

    // ✅ Add student to queue - modified to show proper login alert
    window.addToQueue = function() {
        if (!window.studentName) {
            alert("Failed to join queue: You must be logged in first.");
            return;
        }
        
        // Check if the student already has a pending approval request
        if (localStorage.getItem("pendingApproval") === "true") {
            alert("Failed to join queue: You already have a pending request.");
            return;
        }
        
        const requestData = {
            teacherEmail: teacherName,
            studentName: window.studentName,
        };
        
        const now = new Date();
        const hours = now.getHours();
        const minutes = now.getMinutes().toString().padStart(2, "0");
        const seconds = now.getSeconds().toString().padStart(2, "0");
        const timeIn = `${hours}:${minutes}:${seconds}`;
        localStorage.setItem("timeIn", timeIn);

        // Set flag indicating that this student has a pending approval request
        localStorage.setItem("pendingApproval", "true");

        fetch(approvalURI + "/sendApprovalRequest", {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify(requestData),
        })
        .then(response => {
            if (response.ok) {
                console.log(`✅ Approval request sent for: ${window.studentName}`);
            } else {
                // If request fails, remove the pending flag
                localStorage.removeItem("pendingApproval");
                alert("Failed to request approval.");
            }
        })
        .catch(error => {
            // If request fails, remove the pending flag
            localStorage.removeItem("pendingApproval");
            console.error("Error requesting approval:", error);
            alert("Failed to request approval: Network error.");
        });
    };

    // ✅ Remove student from queue
    window.removeFromQueue = function() {
        if (!window.studentName) {
            alert("Failed to remove from queue: You must be logged in first.");
            return;
        }
        
        const queuer = {
            teacherEmail: teacherName,
            studentName: window.studentName,
        };

        fetch(removeUrl, {
            ...deleteOptions,
            body: JSON.stringify(queuer),
        })
        .then(response => {
            if (response.ok) {
                // Clear the pending approval flag when successfully removed from queue
                localStorage.removeItem("pendingApproval");
                window.fetchQueueData();
            } else {
                alert("Failed to remove from queue.");
            }
        })
        .catch(error => {
            console.error("Error removing from queue:", error);
            alert("Failed to remove from queue: Network error.");
        });
    };

    // ✅ Display queue
    window.displayQueue = function() {
        const studentList = document.getElementById("studentList");
        studentList.innerHTML = ""; // Clear the list

        window.studentsInQueue.forEach((student, index) => {
            const listItem = document.createElement("li");
            
            // Handle potentially null student names
            const studentName = student || "Unknown Student";

            // Indicate waiting for approval for the first student in line
            if (index === 0) {
                listItem.textContent = `${studentName} (waiting for approval)`;
            } else {
                listItem.textContent = studentName;
            }

            if (window.studentName && student === window.studentName) {
                listItem.style.color = "red"; // Highlight user's name in red
            }
            
            studentList.appendChild(listItem);
        });

        // Update position info only if user is logged in
        if (window.studentName) {
            const position = window.studentsInQueue.indexOf(window.studentName) + 1;
            document.getElementById("position").textContent = position > 0 ? 
                `Your position in line: #${position}` : 
                "Your position in line: Not in queue";
            
            document.getElementById("estimatedTime").textContent = position > 0 ? 
                `Estimated time: ~${position * 3} mins` : 
                "Estimated time: N/A";
        } else {
            document.getElementById("position").textContent = "Your position in line: Not logged in";
            document.getElementById("estimatedTime").textContent = "Estimated time: N/A";
        }
    };

    // ✅ Fetch queue data
    window.fetchQueueData = function() {
        fetch(getUrl, fetchOptions)
            .then(response => {
                if (response.status !== 200) {
                    console.error("Failed to fetch queue data.");
                    return;
                }
                return response.json();
            })
            .then(data => {
                const mortensenQueue = data.find(queue => queue.teacherEmail === teacherName);
                if (mortensenQueue) {
                    window.studentsInQueue = mortensenQueue.peopleQueue.split(",");
                    
                    // If user has a pending request but is not in queue, the request might have been denied
                    const hasPendingRequest = localStorage.getItem("pendingApproval") === "true";
                    const isInQueue = window.studentName && window.studentsInQueue.includes(window.studentName);
                    
                    if (hasPendingRequest && !isInQueue) {
                        // Request was likely denied, clear the pending flag
                        localStorage.removeItem("pendingApproval");
                        console.log("Request appears to have been denied or processed");
                    }
                    
                    const isFrontOfQueue = window.studentName && window.studentsInQueue[0] === window.studentName;
                    if (isFrontOfQueue && !window.isApproving) {
                        if (mortensenQueue.away === 1 && isFrontOfQueue && !window.isApproving) {
                            window.location.href = "{{site.baseurl}}/bathroom/hallpass";
                            return;
                        }
                        window.approveStudent();
                    }
                }
                window.displayQueue();
            })
            .catch(error => console.error("Error fetching data:", error));
    };

    // ✅ Approval function
    window.approveStudent = function() {
        // Clear pending approval flag when student is approved
        localStorage.removeItem("pendingApproval");
        
        // Your existing approval code would go here
    };

    // Initial setup - remove previous interval if it exists
    if (window.queueInterval) {
        clearInterval(window.queueInterval);
    }

    // Set up new interval for fetching queue data
    window.queueInterval = setInterval(window.fetchQueueData, 5000);
</script>